In [1]:
import json
from datetime import datetime

In [2]:
pwd

'/home/jay.je/IMspiredStoryTelling/notebooks'

In [3]:
annotation=json.load(open('/home/jay.je/IMspiredStoryTelling/datasets/VIST/val.story-in-sequence.json'))

In [4]:
annotation.keys()

dict_keys(['images', 'info', 'albums', 'type', 'annotations'])

In [5]:
len(annotation['images'])

21048

In [6]:
annotation['images'][0]

{'datetaken': '2007-07-02 03:54:30',
 'license': '5',
 'title': 'Fourth of July prerequisite',
 'text': '',
 'album_id': '72157600601428727',
 'longitude': '0',
 'url_o': 'https://farm2.staticflickr.com/1125/694227468_f6c433d7d8_o.jpg',
 'secret': '0745b37a62',
 'media': 'photo',
 'latitude': '0',
 'id': '694227468',
 'tags': 'family fun fireworks bbq fourthofjuly'}

In [7]:
annotation['albums'][0]

{'description': 'Bubba burgers, beer and BBQ make for a great Fourth of July',
 'title': 'Fourth of July 2007',
 'farm': '2',
 'date_update': '1301274687',
 'primary': '694227468',
 'server': '1125',
 'date_create': '1183374254',
 'photos': '15',
 'secret': '0745b37a62',
 'owner': '67332546@N00',
 'vist_label': '4th_of_july',
 'id': '72157600601428727'}

In [8]:
annotation['annotations'][0]

[{'original_text': 'My sister arrived early to help me with the family Bar BQ.',
  'album_id': '72157600601428727',
  'photo_flickr_id': '693397887',
  'setting': 'first-2-pick-and-tell',
  'worker_id': 'AS2MDFCRUS236TU',
  'story_id': '40470',
  'tier': 'story-in-sequence',
  'worker_arranged_photo_order': 0,
  'text': 'my sister arrived early to help me with the family bar bq .',
  'storylet_id': '202350'}]

## Create "Sents" (sentences from annotations)

In [9]:
sents = []
for ann in annotation['annotations']:
	# sent = {album_id, img_id, story_id, text, original_text, }
	sent = ann[0].copy()
	sent['id'] = sent.pop('storylet_id')
	sent['order'] = sent.pop('worker_arranged_photo_order')
	sent['img_id'] = sent.pop('photo_flickr_id')
	sent['length'] = len(sent['text'].split())  # add length property
	sents += [sent]


In [10]:
annotation['annotations'][0][0]

{'original_text': 'My sister arrived early to help me with the family Bar BQ.',
 'album_id': '72157600601428727',
 'photo_flickr_id': '693397887',
 'setting': 'first-2-pick-and-tell',
 'worker_id': 'AS2MDFCRUS236TU',
 'story_id': '40470',
 'tier': 'story-in-sequence',
 'worker_arranged_photo_order': 0,
 'text': 'my sister arrived early to help me with the family bar bq .',
 'storylet_id': '202350'}

In [11]:
sents[0]

{'original_text': 'My sister arrived early to help me with the family Bar BQ.',
 'album_id': '72157600601428727',
 'setting': 'first-2-pick-and-tell',
 'worker_id': 'AS2MDFCRUS236TU',
 'story_id': '40470',
 'tier': 'story-in-sequence',
 'text': 'my sister arrived early to help me with the family bar bq .',
 'id': '202350',
 'order': 0,
 'img_id': '693397887',
 'length': 13}

## Get Album id to image id mapping

In [12]:
Albums = {album['id']: album for album in annotation['albums']}
Images = {img['id']: img for img in annotation['images']}
Sents = {sent['id']: sent for sent in sents}

In [13]:
def getDateTime(img_id):
	x = Images[img_id]['datetaken']
	return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

In [14]:
album_to_img_ids = {}
for img in annotation['images']:
	album_id = img['album_id']
	img_id = img['id']
	album_to_img_ids[album_id] = album_to_img_ids.get(album_id, []) + [img_id]
for album_id, img_ids in album_to_img_ids.items():
	img_ids.sort(key=getDateTime)

## Get story id -> Sent ids

In [15]:
def get_order(sent_id):
	return Sents[sent_id]['order']

In [16]:
story_to_sent_ids = {}
for sent_id, sent in Sents.items():
	story_id = sent['story_id']
	story_to_sent_ids[story_id] = story_to_sent_ids.get(story_id, []) + [sent_id]
for story_id, sent_ids in story_to_sent_ids.items():
	sent_ids.sort(key=get_order)

In [17]:
story_to_sent_ids # story id to sent id

{'40470': ['202350', '202351', '202352', '202353', '202354'],
 '40471': ['202355', '202356', '202357', '202358', '202359'],
 '40472': ['202360', '202361', '202362', '202363', '202364'],
 '40473': ['202365', '202366', '202367', '202368', '202369'],
 '40474': ['202370', '202371', '202372', '202373', '202374'],
 '40475': ['202375', '202376', '202377', '202378', '202379'],
 '40476': ['202380', '202381', '202382', '202383', '202384'],
 '40477': ['202385', '202386', '202387', '202388', '202389'],
 '40478': ['202390', '202391', '202392', '202393', '202394'],
 '40479': ['202395', '202396', '202397', '202398', '202399'],
 '40480': ['202400', '202401', '202402', '202403', '202404'],
 '40481': ['202405', '202406', '202407', '202408', '202409'],
 '40482': ['202410', '202411', '202412', '202413', '202414'],
 '40483': ['202415', '202416', '202417', '202418', '202419'],
 '40484': ['202420', '202421', '202422', '202423', '202424'],
 '40485': ['202425', '202426', '202427', '202428', '202429'],
 '40486'

## Album id -> Story id

In [18]:
album_to_story_ids = {}
for story_id, sent_ids in story_to_sent_ids.items():
	sent = Sents[sent_ids[0]]
	album_id = sent['album_id']
	album_to_story_ids[album_id] = album_to_story_ids.get(album_id, []) + [story_id]

In [20]:
album_to_story_ids # each album has set of stories, and set of stories map to sent id

{'72157600601428727': ['40470', '40471', '40472', '40473', '40474'],
 '72157601202851033': ['40475', '40476', '40477', '40478', '40479'],
 '72157594187408667': ['40480', '40481', '40482', '40483', '40484'],
 '72157594234060064': ['40485', '40486', '40487', '40488', '40489'],
 '72157600650275265': ['40490', '40491', '40492', '40493', '40494'],
 '72157600650711694': ['40495', '40496', '40497', '40498', '40499'],
 '72157600652875158': ['40500', '40501', '40502', '40503', '40504'],
 '72157600654732083': ['40505', '40506', '40507', '40508', '40509'],
 '72157594269296716': ['40510', '40511', '40512', '40513', '40514'],
 '72157605992804699': ['40515', '40516', '40517', '40518', '40519'],
 '72157600691710127': ['40520', '40521', '40522', '40523', '40524'],
 '72157594192023605': ['40525', '40526', '40527', '40528', '40529'],
 '72157600715772999': ['40530', '40531', '40532', '40533', '40534'],
 '72157600723120795': ['40535', '40536', '40537', '40538', '40539'],
 '72157600725398448': ['40540', '4

In [21]:
for album in annotation['albums']:
	album['img_ids'] = album_to_img_ids[album['id']]
	album['story_ids'] = album_to_story_ids[album['id']]

In [22]:
annotation['albums'][1]

{'description': '',
 'title': 'WMATA - WDC - 04 July 2007',
 'farm': '2',
 'date_update': '1405499691',
 'primary': '1001024065',
 'server': '1232',
 'date_create': '1186184563',
 'photos': '14',
 'secret': 'a94d3a3421',
 'owner': '95413346@N00',
 'vist_label': '4th_of_july',
 'id': '72157601202851033',
 'img_ids': ['727540815',
  '727540891',
  '727541051',
  '727541139',
  '727541385',
  '1003892138',
  '1003892224',
  '1003892334',
  '1003892390',
  '1003892528',
  '1003969840',
  '1003969918',
  '1003969962',
  '1001024065'],
 'story_ids': ['40475', '40476', '40477', '40478', '40479']}

In [26]:
for album in annotation['albums']:
	album['img_ids'] = album_to_img_ids[album['id']]
	album['story_ids'] = album_to_story_ids[album['id']]

# make Stories: {story_id: {id, album_id, sent_ids, img_ids}}
Stories = {story_id: {
                # 'id': story_id, 
                'sent_ids': sent_ids, 
                'img_ids': [Sents[sent_id]['img_id'] for sent_id in sent_ids],
                'album_id': Sents[sent_ids[0]]['album_id']} 
			for story_id, sent_ids in story_to_sent_ids.items()}

In [27]:
Stories

{'40470': {'sent_ids': ['202350', '202351', '202352', '202353', '202354'],
  'img_ids': ['693397887', '695160730', '694227508', '693397865', '694227468'],
  'album_id': '72157600601428727'},
 '40471': {'sent_ids': ['202355', '202356', '202357', '202358', '202359'],
  'img_ids': ['694227468', '694227412', '694227488', '694227508', '693413687'],
  'album_id': '72157600601428727'},
 '40472': {'sent_ids': ['202360', '202361', '202362', '202363', '202364'],
  'img_ids': ['694227468', '694227412', '694227488', '694227508', '693413687'],
  'album_id': '72157600601428727'},
 '40473': {'sent_ids': ['202365', '202366', '202367', '202368', '202369'],
  'img_ids': ['693397887', '695160730', '694227508', '693397865', '694227468'],
  'album_id': '72157600601428727'},
 '40474': {'sent_ids': ['202370', '202371', '202372', '202373', '202374'],
  'img_ids': ['694227468', '694227412', '694227488', '694227508', '693413687'],
  'album_id': '72157600601428727'},
 '40475': {'sent_ids': ['202375', '202376', '

## Manually download images, because one split do not have consecutive images

In [28]:
import requests
from PIL import Image
import random
import os
import shutil
import urllib.request 

from requests.adapters import HTTPAdapter, Retry

In [29]:
# Download 
len(Stories.keys()) # 5000 stories -> choose 1000 stories. Train has chosen 2000 stories

4990

In [30]:
ridx = random.sample(range(int(min(Stories)), int(max(Stories))), 1000) # collect about 1000 stories, corresponding to 5,000 images

In [31]:
img_list = []
stories = []
for i in ridx:
    if str(i) in Stories:
        stories.append(str(i))
        img_list += Stories[str(i)]['img_ids']

In [32]:
print(len(stories))
print(len(img_list))
print(img_list[0])

985
4925
4234776675


In [33]:
Images['4234776675']

{'datetaken': '2009-12-31 13:26:03',
 'license': '6',
 'title': 'Hantaran #2',
 'text': '',
 'album_id': '72157623116920892',
 'longitude': '0',
 'url_o': 'https://farm3.staticflickr.com/2762/4234776675_87fdaba5d1_o.jpg',
 'secret': '9ecd90a1f0',
 'media': 'photo',
 'latitude': '0',
 'id': '4234776675',
 'tags': 'events'}

In [34]:
def make_request(url):
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    response = session.get(url)

    return response
    # print(parsed)

In [35]:
if os.path.exists('/home/jay.je/datasets/VIST/images/val'):
    shutil.rmtree('/home/jay.je/datasets/VIST/images/val')
os.makedirs('/home/jay.je/datasets/VIST/images/val')

In [36]:
for idx, img_num in enumerate(img_list):
    # url = Images[img_num]['url_o']
    # urllib.request.urlretrieve(url, f'/home/jay.je/datasets/VIST/images/{img_num}.jpg')
    try:
        data = requests.get(Images[img_num]['url_o'], timeout=1).content
        with open(f'/home/jay.je/datasets/VIST/images/val/{img_num}.jpeg', 'wb') as handler:
            handler.write(data)
            handler.close()
    except:
        continue
    if idx % 500 == 0:
        print(f'Current processing number is {idx}...\n')

Current processing number is 0...

Current processing number is 500...

Current processing number is 1000...

Current processing number is 1500...

Current processing number is 2000...

Current processing number is 2500...

Current processing number is 3500...

Current processing number is 4000...

Current processing number is 4500...



In [111]:
# check which stories have incomplete images

In [37]:
img_down = os.listdir('/home/jay.je/datasets/VIST/images/val/')

In [38]:
print(len(img_down))
print(len(set(img_down)))

3716
3716


In [39]:
img_down = set(img_down)

In [40]:
img_down

{'46337598.jpeg',
 '20511182.jpeg',
 '4845969115.jpeg',
 '4242266897.jpeg',
 '156532134.jpeg',
 '4656680363.jpeg',
 '10817205774.jpeg',
 '6931925.jpeg',
 '366422094.jpeg',
 '4657340106.jpeg',
 '450950039.jpeg',
 '16657847.jpeg',
 '3897155234.jpeg',
 '65351062.jpeg',
 '4394048781.jpeg',
 '26871635.jpeg',
 '4282409651.jpeg',
 '6561216.jpeg',
 '6119703333.jpeg',
 '135822685.jpeg',
 '4422859573.jpeg',
 '16025977.jpeg',
 '4287803.jpeg',
 '371626042.jpeg',
 '21155918.jpeg',
 '4970986830.jpeg',
 '180884943.jpeg',
 '74921380.jpeg',
 '761017048.jpeg',
 '19654794.jpeg',
 '4361810743.jpeg',
 '11964239745.jpeg',
 '26371911.jpeg',
 '29300019.jpeg',
 '4314090686.jpeg',
 '5387726188.jpeg',
 '53435420.jpeg',
 '219689.jpeg',
 '4766206.jpeg',
 '21692207.jpeg',
 '349122586.jpeg',
 '4283666240.jpeg',
 '5150761608.jpeg',
 '386831711.jpeg',
 '550214346.jpeg',
 '5012962957.jpeg',
 '407244768.jpeg',
 '4919305094.jpeg',
 '349303158.jpeg',
 '20427686.jpeg',
 '449845527.jpeg',
 '4205442748.jpeg',
 '50152068.jpeg

In [41]:
drop_stories=[]
for i in Stories:
    img_ids = Stories[i]['img_ids']
    for j in img_ids:
        if j+'.jpeg' not in img_down:
            drop_stories.append(i)
            break

In [42]:
drop_stories = set(drop_stories)
print(len(drop_stories))

2767


In [43]:
set(Stories.keys())-drop_stories

{'42068',
 '45194',
 '45499',
 '44574',
 '42497',
 '44422',
 '40660',
 '45008',
 '43393',
 '41009',
 '40664',
 '42465',
 '43412',
 '45184',
 '44997',
 '43812',
 '40931',
 '40657',
 '42109',
 '41147',
 '44115',
 '44817',
 '44776',
 '44262',
 '41470',
 '42392',
 '43755',
 '41953',
 '41131',
 '44229',
 '42243',
 '42157',
 '41438',
 '42219',
 '40885',
 '41182',
 '42738',
 '44298',
 '44142',
 '41701',
 '40801',
 '42821',
 '43944',
 '44964',
 '43621',
 '44519',
 '44047',
 '45300',
 '43302',
 '44480',
 '43684',
 '42953',
 '40842',
 '43932',
 '43418',
 '41538',
 '40970',
 '40643',
 '45014',
 '40676',
 '42547',
 '44658',
 '41473',
 '42973',
 '42986',
 '44180',
 '42625',
 '43863',
 '41430',
 '43450',
 '43805',
 '42721',
 '42914',
 '40774',
 '44348',
 '43875',
 '41359',
 '42287',
 '40950',
 '42193',
 '43814',
 '44369',
 '43759',
 '43623',
 '43810',
 '44245',
 '42795',
 '42375',
 '41738',
 '44036',
 '41123',
 '45280',
 '43360',
 '41598',
 '45250',
 '42278',
 '42367',
 '45418',
 '43245',
 '44998',


In [44]:
len(set(Stories.keys())-drop_stories)

2223

In [45]:
ValStoriesFin = {}
for i in Stories:
    if i not in drop_stories:
        ValStoriesFin[i] = Stories[i].copy()

In [46]:
len(ValStoriesFin.keys()) # able to extract 2269 stories from this!

2223

In [48]:
for s in ValStoriesFin:
    for i in ValStoriesFin[s]['sent_ids']:
        if 'text' not in ValStoriesFin[s]:
            ValStoriesFin[s]['text'] = []
        ValStoriesFin[s]['text'].append(Sents[i]['text'])

## Save the final files

In [51]:
sents_json = json.dumps(Sents, indent=4)
# Writing to sample.json
with open("/home/jay.je/IMspiredStoryTelling/datasets/VIST/val/annotations/ValSents.json", "w") as outfile:
    outfile.write(sents_json)
    outfile.close()

In [53]:
storiesfin_json = json.dumps(ValStoriesFin, indent=4)
# Writing to sample.json
with open("/home/jay.je/IMspiredStoryTelling/datasets/VIST/val/annotations/ValStoriesFin.json", "w") as outfile:
    outfile.write(storiesfin_json)
    outfile.close()

In [54]:
stories_json = json.dumps(Stories, indent=4)
# Writing to sample.json
with open("/home/jay.je/IMspiredStoryTelling/datasets/VIST/val/annotations/ValStories.json", "w") as outfile:
    outfile.write(stories_json)
    outfile.close()